In [1]:
from pathlib import Path
import cv2
import sys
import numpy as np
from skimage import feature, measure
import live_cell_tracker
from live_cell_tracker.livecell_segment import datasets
import live_cell_tracker.livecell_segment
from live_cell_tracker import livecell_segment
import live_cell_tracker.core.utils

seg_path = r"""D:\xing-vimentin-dic-pipeline\src\cxa_segmentation\notebooks\segmentation_results\detectron_model3\restart_day0_Group 1_wellA1_RI_MIP_stitched"""
label_seg_imgs = livecell_segment.datasets.LiveCellImageDataset(seg_path, ext="png")
dir_path = Path(
    r"D:\xing-vimentin-dic-pipeline\src\cxa_segmentation\cxa-data\june_2022_data_8bit_png\restart_day0_Group 1_wellA1_RI_MIP_stitched"
)
raw_imgs = livecell_segment.datasets.LiveCellImageDataset(dir_path, ext="png")

576 png img file paths loaded: 
288 png img file paths loaded: 


In [2]:
def gen_detections_for_sort_tracker(label_mask):
    """
        generate detections for SORT tracker. detections: [x1, y1, x2, y2, score]
    Parameters
    ----------
    label_mask :
        an image
    Returns
    -------
    A list of (x1, y1, x2, y2, score]) for each object detected
    """    
    regionprops_skimge = measure.regionprops(label_mask)
    boxes = live_cell_tracker.core.utils.get_bbox_from_regionprops(regionprops_skimge)
    detections = np.array([list(bbox) + [1] for bbox in boxes])
    return detections

import live_cell_tracker.track.sort_tracker
tracker = live_cell_tracker.track.sort_tracker.Sort() 

all_track_bbs = []
for i, img in enumerate(label_seg_imgs):
    detections = gen_detections_for_sort_tracker(img)
    track_bbs_ids = tracker.update(detections)
    all_track_bbs.append(track_bbs_ids)
    # for testing
    if i > 5:
        break

In [3]:
all_track_bbs[0]

array([[1.000e+00, 1.396e+03, 6.700e+01, 1.581e+03, 2.700e+01],
       [7.340e+02, 2.114e+03, 9.140e+02, 2.264e+03, 2.600e+01],
       [3.820e+02, 2.740e+02, 4.690e+02, 3.900e+02, 2.500e+01],
       [1.385e+03, 1.310e+03, 1.445e+03, 1.363e+03, 2.400e+01],
       [1.395e+03, 1.278e+03, 1.450e+03, 1.323e+03, 2.300e+01],
       [1.829e+03, 1.590e+03, 1.906e+03, 1.619e+03, 2.200e+01],
       [1.440e+02, 1.333e+03, 2.310e+02, 1.391e+03, 2.100e+01],
       [2.850e+02, 2.700e+02, 3.420e+02, 3.580e+02, 2.000e+01],
       [6.000e+01, 1.266e+03, 1.480e+02, 1.329e+03, 1.900e+01],
       [9.000e+01, 1.414e+03, 1.660e+02, 1.473e+03, 1.800e+01],
       [1.850e+03, 1.369e+03, 1.904e+03, 1.419e+03, 1.700e+01],
       [1.200e+01, 1.234e+03, 3.010e+02, 1.400e+03, 1.600e+01],
       [2.710e+02, 4.650e+02, 3.510e+02, 5.290e+02, 1.500e+01],
       [7.000e+00, 1.234e+03, 1.150e+02, 1.304e+03, 1.400e+01],
       [7.600e+02, 2.236e+03, 9.870e+02, 2.377e+03, 1.300e+01],
       [1.012e+03, 2.321e+03, 1.102e+03,

convert to int

In [4]:
all_track_bbs = [track_bbs.astype(np.int32) for track_bbs in all_track_bbs]
all_track_bbs[0]

array([[   1, 1396,   67, 1581,   27],
       [ 734, 2114,  914, 2264,   26],
       [ 382,  274,  469,  390,   25],
       [1385, 1310, 1445, 1363,   24],
       [1395, 1278, 1450, 1323,   23],
       [1829, 1590, 1906, 1619,   22],
       [ 144, 1333,  231, 1391,   21],
       [ 285,  270,  342,  358,   20],
       [  60, 1266,  148, 1329,   19],
       [  90, 1414,  166, 1473,   18],
       [1850, 1369, 1904, 1419,   17],
       [  12, 1234,  301, 1400,   16],
       [ 271,  465,  351,  529,   15],
       [   7, 1234,  115, 1304,   14],
       [ 760, 2236,  987, 2377,   13],
       [1012, 2321, 1102, 2385,   12],
       [ 103, 1304,  322, 1406,   11],
       [1733, 1571, 1989, 1690,   10],
       [ 988, 2282, 1140, 2402,    9],
       [ 780, 2238,  821, 2253,    8],
       [1783, 1494, 2034, 1574,    7],
       [  69,  424,  607,  694,    6],
       [ 350,  180,  584,  413,    5],
       [1789, 1256, 2015, 1480,    4],
       [  48, 1391,  280, 1543,    3],
       [1297, 1159, 1571,

In [5]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
idx = 2
bboxes = all_track_bbs[idx]
fig, ax = plt.subplots()
ax.imshow(raw_imgs[idx])
for bbox in bboxes:
    # Create a Rectangle patch
    rect = patches.Rectangle((bbox[1], bbox[0]),  (bbox[3] - bbox[1]), (bbox[2] - bbox[0]), linewidth=1, edgecolor='r', facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)
plt.show()


KeyboardInterrupt: 

In [6]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os

def visualize_bbox_trajectory(track_id, all_track_bbs, raw_imgs, save_path="./tmp.gif", track_id_col = 4):
    fig, ax = plt.subplots()
    def init():
        return []
    def update(frame):
        frame_idx, raw_img, bbox, img_crop = frame
        ax.cla()
        frame_text = ax.text(-10, -10, "frame: {}".format(frame_idx), fontsize=10, color="red", ha="center", va="center")
        ax.imshow(img_crop)
        return []

    frame_data = []
    for frame_idx, track_bbs in enumerate(all_track_bbs):
        img = raw_imgs[frame_idx]
        for bbox in track_bbs:
            if bbox[track_id_col] != track_id:
                continue
            min_x, max_x, min_y, max_y = int(bbox[0]), int(bbox[2]), int(bbox[1]), int(bbox[3])
            img_crop = img[min_x:max_x, min_y:max_y]
            frame_data.append((frame_idx, img, bbox, img_crop))

    ani = FuncAnimation(fig, update, frames=frame_data,
                init_func=None, blit=True)
    print("saving to: %s..." % save_path)
    ani.save(save_path)

output_path = Path("./track_tmp")
output_path.mkdir(exist_ok=True)

handled_track_ids = set()
track_start_end = {

}
for i, objects in enumerate(all_track_bbs):
    for obj in objects:
        track_id = obj[-1]
        if  track_id in handled_track_ids:
            continue
        handled_track_ids.add(track_id)
        visualize_bbox_trajectory(track_id, all_track_bbs, raw_imgs, save_path=output_path / "track_{}.gif".format(track_id))



MovieWriter ffmpeg unavailable; using Pillow instead.


saving to: track_tmp\track_27.gif...


MovieWriter ffmpeg unavailable; using Pillow instead.


saving to: track_tmp\track_26.gif...


MovieWriter ffmpeg unavailable; using Pillow instead.


...

saving to: track_tmp\track_31.gif...


MovieWriter ffmpeg unavailable; using Pillow instead.


saving to: track_tmp\track_30.gif...


MovieWriter ffmpeg unavailable; using Pillow instead.


saving to: track_tmp\track_29.gif...
